<a href="https://colab.research.google.com/github/nayrr25/Aplicaciones-Financieras-e-IA/blob/main/BERT_examen3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fase 1: Importar las dependencias

In [1]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random

from google.colab import drive

In [2]:
!pip install bert-for-tf2
!pip install sentencepiece

In [3]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


# Fase 2: Pre procesado de datos

## Carga de los ficheros

Importamos los ficheros desde nuestro Google Drive personal.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd

# Asegúrate de usar la ruta correcta a tu archivo
file_path = '/content/drive/MyDrive/AplicacionesFinancieras/DataAnalyst.csv'

df = pd.read_csv(file_path)

In [6]:
X= df[['Job Title', 'Job Description','Sector']]
y= df[['Salary Estimate']]

In [7]:
X.head(6)

,Job Title,Job Description,Sector
0,"Data Analyst, Center on Immigration and Justic...",Are you eager to roll up your sleeves and harn...,Non-Profit
1,Quality Data Analyst,Overview\n\nProvides analytical and technical ...,Health Care
2,"Senior Data Analyst, Insights & Analytics Team...",We’re looking for a Senior Data Analyst who ha...,Information Technology
3,Data Analyst,Requisition NumberRR-0001939\nRemote:Yes\nWe c...,Information Technology
4,Reporting Data Analyst,ABOUT FANDUEL GROUP\n\nFanDuel Group is a worl...,"Arts, Entertainment & Recreation"
5,Data Analyst,About Cubist\nCubist Systematic Strategies is ...,Finance


## Preprocesar la información

### Limpieza

#### convertir el df en dataframe


In [8]:
df = pd.DataFrame(df)

# Limpiar el campo de estimación salarial
df["Salary Estimate"] = df["Salary Estimate"].str.replace(r" \(Glassdoor est.\)", "", regex=True)

In [9]:
# Ejemplo de normalización y conversión en variable numérica para la predicción
df["Salary Estimate"] = df["Salary Estimate"].str.replace('K', '000')


In [55]:
# conversión de X en data_clean, sumando cada uno de los vectores
data_clean= df['Job Title']+ " " + df['Job Description']+ " " + df['Sector']

In [57]:
df["data_clean"]= data_clean

In [12]:

# Convertir rangos a valores numéricos y calcular el promedio
df['min_salary'] = df['Salary Estimate'].str.split('-').str[0].str.replace('K', '').str.replace('$', '').astype(int) * 1000
df['max_salary'] = df['Salary Estimate'].str.split('-').str[1].str.replace('K', '').str.replace('$', '').astype(int) * 1000
df['avg_salary'] = (df['min_salary'] + df['max_salary']) / 2

# Mostrar las primeras filas después de la conversión para verificar
#print("Primeras filas después de la conversión:", df[['Salary Estimate', 'min_salary', 'max_salary', 'avg_salary']].head())


In [13]:
df['avg_salary']

0       51500000.0
1       51500000.0
2       51500000.0
3       51500000.0
4       51500000.0
          ...     
257    100000000.0
258    100000000.0
259    100000000.0
260    100000000.0
261    100000000.0
Name: avg_salary, Length: 262, dtype: float64

In [14]:
# de las tres posibilidades, uitilizaré el promedio

In [23]:
data_labels = df["avg_salary"].values
# Convert to a pandas Series
data_labels = pd.Series(data_labels)
pd.set_option('display.float_format', '{:.2f}'.format)
print(data_labels)

0      51500000.00
1      51500000.00
2      51500000.00
3      51500000.00
4      51500000.00
          ...     
257   100000000.00
258   100000000.00
259   100000000.00
260   100000000.00
261   100000000.00
Length: 262, dtype: float64


### Tokenization

Necesitaremos crear una capa BERT para tener acceso a los meta datos para el tokenizador (como el tamaño del vocabulario).

### Paso 1: Importar las bibliotecas necesarias
Asegúrate de que tienes las bibliotecas necesarias importadas para ejecutar el código, como tensorflow_hub y bert-for-tf2:

In [46]:
import tensorflow_hub as hub
import tensorflow as tf
#!pip install bert-for-tf2
import bert


### Paso 2: Cargar la Capa BERT desde TensorFlow Hub
El código carga un modelo BERT (bert_en_uncased_L-12_H-768_A-12) como una capa de Keras de TensorFlow Hub, que no será entrenable (trainable=False). Esto es útil para tareas que solo requieren la representación de BERT sin ajustar sus pesos:

In [47]:
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)

### Paso 3: Preparar el Tokenizer
Se prepara el tokenizer utilizando la información del modelo cargado:

Archivo de Vocabulario: vocab_file se obtiene directamente del modelo BERT cargado, que contiene el vocabulario usado durante el entrenamiento del modelo.
Case Sensitivity: do_lower_case indica si el modelo fue entrenado con textos en minúsculas, lo cual es importante para saber cómo procesar el texto de entrada.

In [48]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = bert.bert_tokenization.FullTokenizer(vocab_file, do_lower_case)

### Paso 4: Utilizar el Tokenizer
Una vez que tienes el tokenizer configurado, puedes usarlo para tokenizar tus textos. Por ejemplo:

In [49]:
text = "Here is an example sentence to tokenize using BERT tokenizer."
tokens = tokenizer.tokenize(text)
print(tokens)

['here', 'is', 'an', 'example', 'sentence', 'to', 'token', '##ize', 'using', 'bert', 'token', '##izer', '.']


### Paso 5: Aplicar el Tokenizer
Una vez que tienes la columna data_clean lista, puedes aplicar el tokenizer a cada entrada de esta columna. Usaremos una comprensión de lista para aplicar el tokenizer a cada fila del DataFrame.

In [58]:
# Suponiendo que 'tokenizer' ya está inicializado como mostraste anteriormente
df['tokens'] = df['data_clean'].apply(lambda x: tokenizer.tokenize(x))

In [60]:
df['tokens']

0      [data, analyst, ,, center, on, immigration, an...
1      [quality, data, analyst, overview, provides, a...
2      [senior, data, analyst, ,, insights, &, analyt...
3      [data, analyst, re, ##qui, ##sit, ##ion, numbe...
4      [reporting, data, analyst, about, fan, ##due, ...
                             ...                        
257    [data, analyst, -, qc, nes, ##co, resource, is...
258    [people, operations, &, data, analyst, job, de...
259    [lead, data, analyst, (, product, ), a, bit, a...
260    [data, analyst, -, iii, direct, client, requir...
261    [sql, data, analyst, job, title, :, senior, sq...
Name: tokens, Length: 262, dtype: object

Este código añadirá una nueva columna al DataFrame llamada 'tokens', donde cada entrada es la lista de tokens resultantes de aplicar el tokenizer BERT a la cadena correspondiente en 'data_clean'.

### Paso 6: Verificación
Es una buena práctica verificar que el proceso ha funcionado como se esperaba. Puedes inspeccionar las primeras filas del DataFrame para asegurarte de que los tokens se han generado correctamente:

In [59]:
print(df[['data_clean', 'tokens']].head())

                                          data_clean  \
0  Data Analyst, Center on Immigration and Justic...   
1  Quality Data Analyst Overview\n\nProvides anal...   
2  Senior Data Analyst, Insights & Analytics Team...   
3  Data Analyst Requisition NumberRR-0001939\nRem...   
4  Reporting Data Analyst ABOUT FANDUEL GROUP\n\n...   

                                              tokens  
0  [data, analyst, ,, center, on, immigration, an...  
1  [quality, data, analyst, overview, provides, a...  
2  [senior, data, analyst, ,, insights, &, analyt...  
3  [data, analyst, re, ##qui, ##sit, ##ion, numbe...  
4  [reporting, data, analyst, about, fan, ##due, ...  


### Paso 7: tokenizar las palabras

La función encode_sentence que mencionas está diseñada para convertir un texto de entrada en una lista de identificadores numéricos (IDs) correspondientes a los tokens en el vocabulario de BERT. Este proceso es una parte crucial en la preparación de datos para modelos basados en BERT, ya que estos modelos no trabajan directamente con texto plano sino con tokens que son convertidos en IDs numéricos.

In [63]:
def encode_sentence(sent):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sent))

### Paso 8: Conversión de Tokens a IDs
Una vez que el texto ha sido dividido en tokens, el siguiente paso es convertir cada token en su correspondiente ID numérico:

In [64]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

# Creación del data set

Crearemos padded batches (por lo que rellenamos las frases para cada lote de forma independiente), de esta forma añadimos el mínimo número de tokens de padding posible. Para eso, ordenamos las frases por longitud, aplicamos padded_batches y luego las mezclamos.

In [67]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key=lambda x: x[2])
sorted_all = [(sent_lab[0], sent_lab[1])
              for sent_lab in data_with_len if sent_lab[2] > 7]

In [68]:
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [69]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))

In [70]:
NB_BATCHES = math.ceil(len(sorted_all) / BATCH_SIZE)
NB_BATCHES_TEST = NB_BATCHES // 10
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

In [71]:
test_dataset

<_TakeDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [72]:
train_dataset

<_SkipDataset element_spec=(TensorSpec(shape=(None, None), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

# Fase 3: Construcción del modelo

In [73]:
#DCNN que hereda de tf.keras.Model de TensorFlow Keras API, diseñada específicamente para trabajar con datos textuales,
#probablemente para tareas como clasificación de texto.
class DCNN(tf.keras.Model):
  #El método __init__ inicializa una nueva instancia de DCNN. Aquí es donde configuras todas las capas y parámetros que necesitará tu modelo:
    def __init__(self,
                 vocab_size,
                 emb_dim=128,
                 nb_filters=50,
                 FFN_units=512,
                 nb_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="dcnn"):
        super(DCNN, self).__init__(name=name)

        self.embedding = layers.Embedding(vocab_size,
                                          emb_dim)
        self.bigram = layers.Conv1D(filters=nb_filters,
                                    kernel_size=2,
                                    padding="valid",
                                    activation="relu")
        self.trigram = layers.Conv1D(filters=nb_filters,
                                     kernel_size=3,
                                     padding="valid",
                                     activation="relu")
        self.fourgram = layers.Conv1D(filters=nb_filters,
                                      kernel_size=4,
                                      padding="valid",
                                      activation="relu")
        self.pool = layers.GlobalMaxPool1D()

        self.dense_1 = layers.Dense(units=FFN_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if nb_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=nb_classes,
                                           activation="softmax")

    def call(self, inputs, training):
        x = self.embedding(inputs)
        x_1 = self.bigram(x) # (batch_size, nb_filters, seq_len-1)
        x_1 = self.pool(x_1) # (batch_size, nb_filters)
        x_2 = self.trigram(x) # (batch_size, nb_filters, seq_len-2)
        x_2 = self.pool(x_2) # (batch_size, nb_filters)
        x_3 = self.fourgram(x) # (batch_size, nb_filters, seq_len-3)
        x_3 = self.pool(x_3) # (batch_size, nb_filters)

        merged = tf.concat([x_1, x_2, x_3], axis=-1) # (batch_size, 3 * nb_filters)
        merged = self.dense_1(merged)
        merged = self.dropout(merged, training)
        output = self.last_dense(merged)

        return output

# Fase 4: Entrenamiento

In [74]:
VOCAB_SIZE = len(tokenizer.vocab)
EMB_DIM = 200
NB_FILTERS = 100
FFN_UNITS = 256
NB_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [75]:
Dcnn = DCNN(vocab_size=VOCAB_SIZE,
            emb_dim=EMB_DIM,
            nb_filters=NB_FILTERS,
            FFN_units=FFN_UNITS,
            nb_classes=NB_CLASSES,
            dropout_rate=DROPOUT_RATE)

In [76]:
if NB_CLASSES == 2:
    Dcnn.compile(loss="binary_crossentropy",
                 optimizer="adam",
                 metrics=["accuracy"])
else:
    Dcnn.compile(loss="sparse_categorical_crossentropy",
                 optimizer="adam",
                 metrics=["sparse_categorical_accuracy"])

In [77]:
checkpoint_path = "./drive/My Drive/Curso de NLP/BERT/ckpt_bert_tok/"

ckpt = tf.train.Checkpoint(Dcnn=Dcnn)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Último checkpoint restaurado!!")

Último checkpoint restaurado!!


In [78]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint guardado en {}.".format(checkpoint_path))

In [79]:
Dcnn.fit(train_dataset,
         epochs=NB_EPOCHS,
         callbacks=[MyCustomCallback()])

Epoch 1/5
      9/Unknown - 3s 109ms/step - loss: -11822012825600.0000 - accuracy: 0.0000e+00Checkpoint guardado en ./drive/My Drive/Curso de NLP/BERT/ckpt_bert_tok/.
9/9 [==============================] - 4s 148ms/step - loss: -11822012825600.0000 - accuracy: 0.0000e+00
Epoch 2/5
9/9 [==============================] - 1s 133ms/step - loss: -14944920666112.0000 - accuracy: 0.0000e+00
Epoch 3/5
9/9 [==============================] - 0s 56ms/step - loss: -18704372858880.0000 - accuracy: 0.0000e+00
Epoch 4/5
9/9 [==============================] - 1s 60ms/step - loss: -23100999598080.0000 - accuracy: 0.0000e+00
Epoch 5/5
9/9 [==============================] - 0s 53ms/step - loss: -28347599945728.0000 - accuracy: 0.0000e+00


# Fase 5: Evaluación

In [80]:
# Ejemplo: Si tienes 1000 ejemplos y un tamaño de lote de 32
total_test_samples = 1000
batch_size = 32

steps_per_epoch = total_test_samples // batch_size
if total_test_samples % batch_size != 0:
    steps_per_epoch += 100  # Añadir un paso adicional si hay un remanente

results = Dcnn.evaluate(test_dataset, steps=steps_per_epoch)
print(results)


131/131 [==============================] - 0s 1ms/step
[]


In [85]:
results

[]



*   Training: 88.5%
*   Testing: 84.6%



In [82]:
def get_prediction(sentence):
    tokens = encode_sentence(sentence)
    inputs = tf.expand_dims(tokens, 0)

    output = Dcnn(inputs, training=False)

    sentiment = math.floor(output*2)

    if sentiment == 0:
        print("Salida del modelo: {}\nSentimiento predicho: Negativo.".format(
            output))
    elif sentiment == 1:
        print("Salida del modelo: {}\nSentimiento predicho: Positivo.".format(
            output))

In [83]:
get_prediction("This movie was pretty interesting.")

In [84]:
get_prediction("I'd rather not do that again.")